## Similarity and distance metrics

scikit-fingerprints offers plenty of ways to measure similarity between molecules.

In general, we can divide similarity metrics into two groups:
 - those working on the vectorized molecules, which can be further divided based on the type of input:
     - binary data,
     - count (non-binary) data.
 - those working on RDKit's ``Mol`` objects.

Additionally, every metric is also implemented in a bulk form, allowing matrices to be passed as input.
Every similarity has its corresponding distance, which is usually equal to $1 - similarity$. 
However, there are some exceptions (see next section).

# Metrics overview

Scikit-Fingerprints currently supports 14 different metrics.

Vector-based methods can be split into two groups. The first considers only 'on' bits (1s in vector representation), and the second also takes 'off' bits (0s in vector representation) into consideration.

Keep in mind that metrics use a slightly different naming convention. If a similarity function takes arguments
$x$ and $y$, then inside the formula:

  - $a$ – $|x \cap y|$, the number of common "on" bits
  - $b$ – the number of positions where $x$ is 1 and $y$ is 0
  - $c$ – the number of positions where $x$ is 0 and $y$ is 1
  - $d$ – the number of positions where both are 0

Let's see a couple of examples.

**Both "on" and "off" bits**:

 - [Rogot-Goldberg](https://doi.org/10.1016/0021-9681(66)90032-4): 
   - binary: $sim(x, y) = \frac{a}{2 * (2a + b + c)} + \frac{d}{2 * (2d + b + c)}$,
   - values in range $(0, 1)$,

 - [Tanimoto](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-015-0069-3):
   - binary: $sim(a, b) = \frac{|a \cap b|}{|a \cup b|} = \frac{|a \cap b|}{|a| + |b| - |a \cap b|}$
   - count: $sim(a, b) = \frac{a \cdot b}{\|a\|^2 + \|b\|^2 - a \cdot b}$,
   - values in range $(0, 1)$,

**Only "on" bits**:

 - [Rand](https://www.tandfonline.com/doi/abs/10.1080/01621459.1971.10482356):
   - binary: $sim(a, b) = \frac{|a \cap b|}{n}$,
   - values in range $(0, 1)$,

 - [Russell](https://www.cabidigitallibrary.org/doi/full/10.5555/19412900343):
   - binary: $sim(x, y) = \frac{a}{n}$,
   - values in range $(0, 1)$,


**Operating on mols**:

 - [Fraggle](https://raw.github.com/rdkit/UGM_2013/master/Presentations/Hussain.Fraggle.pdf):
   - uses fragmentation algorithm (ring cut in this case) to split the **query molecule** at acyclic and ring bonds,
   - fragments are compared to the **reference molecule** using [Tversky similarity](https://en.wikipedia.org/wiki/Tversky_index) (α=0.95,
    β=0.05),
   - fragments with **Tversky similarity** above defined threshold are used for calculating Tanimoto similarity with reference molecule,
   - values in range $(0, 1)$
 - [MCS](https://doi.org/10.1007/s10822-015-9872-1) (Maximum Common Similarity):
   - uses [FMCS algorithm](https://doi.org/10.1186/1758-2946-5-S1-O6) to measure overlap of two molecules,
   - penalizes difference in number of atoms between molecules,
   - $sim(mol_a, mol_b) = \frac{numAtoms(MCS(mol_a, mol_b))}{numAtoms(mol_a) + numAtoms(mol_b) - numAtoms(MCS(mol_a, mol_b))}$
   - values in range $(0, 1)$

In this tutorial, we'll use Tanimoto and Fraggle metrics as examples.

## Binary similarity

We can pass the input data as either numpy  or csr sparse arrays

In [40]:
import numpy as np
from scipy.sparse import csr_array

vec_a = [1, 0, 0]
vec_b = [0, 1, 1]

vec_a_numpy = np.array(vec_a)
vec_b_numpy = np.array(vec_b)

vec_a_sparse = csr_array([vec_a])
vec_b_sparse = csr_array([vec_b])

The distance function will yield the same result, no matter the format

In [41]:
from skfp.distances.tanimoto import tanimoto_binary_distance, tanimoto_binary_similarity

binary_sim_numpy = tanimoto_binary_similarity(vec_a_numpy, vec_b_numpy)
binary_dist_numpy = tanimoto_binary_distance(vec_a_numpy, vec_b_numpy)

binary_sim_sparse = tanimoto_binary_similarity(vec_a_sparse, vec_b_sparse)
binary_dist_sparse = tanimoto_binary_distance(vec_a_sparse, vec_b_sparse)

In [45]:
# ruff: noqa: S101
assert binary_sim_numpy == binary_sim_sparse == 0

In [47]:
assert binary_dist_numpy == binary_dist_sparse == 1

In [48]:
assert binary_dist_numpy == 1 - binary_sim_numpy
assert binary_dist_sparse == 1 - binary_sim_sparse

## Count similarity

Count variants work exactly the same way as binary ones.

In [49]:
import numpy as np
from scipy.sparse import csr_array

from skfp.distances.tanimoto import tanimoto_count_distance, tanimoto_count_similarity

vec_a = [2, 3, 4, 0]
vec_b = [2, 3, 4, 2]

vec_a_numpy = np.array(vec_a)
vec_b_numpy = np.array(vec_b)

vec_a_sparse = csr_array([vec_a])
vec_b_sparse = csr_array([vec_b])

In [50]:
count_sim_numpy = tanimoto_count_similarity(vec_a_numpy, vec_b_numpy)
count_dist_numpy = tanimoto_count_distance(vec_a_numpy, vec_b_numpy)

count_sim_sparse = tanimoto_count_similarity(vec_a_sparse, vec_b_sparse)
count_dist_sparse = tanimoto_count_distance(vec_a_sparse, vec_b_sparse)

In [51]:
assert count_sim_numpy == count_sim_sparse == 0.8787878787878788

In [52]:
assert count_dist_numpy == count_dist_sparse == 0.12121212121212122

In [53]:
assert count_dist_numpy == 1 - count_sim_numpy
assert count_dist_sparse == 1 - count_sim_sparse

## Structural similarity

In this case, we use RDKit's mols rather than vectorized data. First, prepare the query and reference molecules.

In [ ]:
from rdkit.Chem import MolFromSmiles

mol_query = MolFromSmiles("COc1cc(CN2CCC(NC(=O)c3cncc(C)c3)CC2)c(OC)c2ccccc12")
mol_ref = MolFromSmiles("COc1ccccc1")

Let's call fraggle function. The default Tversky threshold is 0.8.

In [13]:
from skfp.distances.fraggle import fraggle_distance, fraggle_similarity

fraggle_sim = fraggle_similarity(mol_query, mol_ref)
fraggle_dist = fraggle_distance(mol_query, mol_ref)

Values fall within range $(0, 1)$, so we expect the distance to be $1 - similarity$.

In [14]:
assert fraggle_sim == 0.1640625
assert fraggle_dist == 0.8359375
assert fraggle_dist == 1 - fraggle_sim

## kNN

We aim to make SKFP fully compatible with Scikit-Learn. As such, you can use similarity metrics in your machine learning pipelines, for example, as a metric in the k-Nearest Neighbors algorithm.

Prepare the data and compute the distance using the SKFP function.

In [55]:
from sklearn.neighbors import NearestNeighbors

vec_a = np.array([[0, 1, 0, 1]])
vec_b = np.array([[0, 1, 1, 0]])
skfp_dist = tanimoto_binary_distance(vec_a[0], vec_b[0])

skfp_dist

0.6666666666666667

Now, let's compare the result with Scikit-Learn's kNN implementation utilizing the SKFP metric.

In [58]:
nn = NearestNeighbors(n_neighbors=1, metric=tanimoto_binary_distance)
nn.fit(vec_a)
distances, _ = nn.kneighbors(vec_b)

distances

array([[0.66666667]])

NearestNeighbors returns a matrix, so we'll need to extract a single value for comparison.

In [59]:
sklearn_dist = distances[0][0]
assert np.isclose(skfp_dist, sklearn_dist)

## Bulk variants

If you need to quickly compute similarity or distance between many vectors, you may represent them as an array and pass it to **bulk** variant of a function. In the example below, the similarity is computed between i-th rows and j-th columns of both arrays.

Keep in mind that bulk variants do not support sparse arrays, as they are not valid data type for Numba, which we use for accelerating the computations.

Bulk variants are equivalent to scikit-learn's **pairwise distances**.

In [16]:
X = np.array(
    [
        [1, 1, 1],
        [0, 0, 1],
        [1, 1, 1],
    ]
)

Y = np.array(
    [
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ]
)

X_numpy = np.array(X)
Y_numpy = np.array(Y)

In [17]:
from skfp.distances.tanimoto import (
    bulk_tanimoto_binary_distance,
    bulk_tanimoto_binary_similarity,
)

In [18]:
bulk_binary_sim_numpy = bulk_tanimoto_binary_similarity(X_numpy, Y_numpy)
bulk_binary_dist_numpy = bulk_tanimoto_binary_distance(X_numpy, Y_numpy)

In [19]:
bulk_binary_sim_numpy

array([[0.66666667, 0.66666667, 1.        ],
       [0.5       , 0.5       , 0.33333333],
       [0.66666667, 0.66666667, 1.        ]])

In [20]:
bulk_binary_dist_numpy

array([[0.33333333, 0.33333333, 0.        ],
       [0.5       , 0.5       , 0.66666667],
       [0.33333333, 0.33333333, 0.        ]])

Let's make sure that the distance is, in fact, $1−similarity$. Subtracting the similarity matrix from a scalar may look odd, but NumPy handles broadcasting nicely.

In [21]:
assert (np.isclose(bulk_binary_dist_numpy, 1 - bulk_binary_sim_numpy)).all()

You can also use the count Tanimoto variant.

In [22]:
X = np.array(
    [
        [1, 2, 1],
        [0, 1, 3],
        [7, 1, 5],
    ]
)

Y = np.array(
    [
        [9, 0, 3],
        [8, 4, 1],
        [1, 2, 1],
    ]
)

X_numpy = np.array(X)
Y_numpy = np.array(Y)

In [23]:
from skfp.distances.tanimoto import (
    bulk_tanimoto_count_distance,
    bulk_tanimoto_count_similarity,
)

In [24]:
bulk_count_sim_numpy = bulk_tanimoto_count_similarity(X_numpy, Y_numpy)
bulk_count_dist_numpy = bulk_tanimoto_count_distance(X_numpy, Y_numpy)

In [25]:
bulk_count_sim_numpy

array([[0.14285714, 0.24285714, 1.        ],
       [0.0989011 , 0.08333333, 0.45454545],
       [0.89655172, 0.71428571, 0.20895522]])

In [26]:
bulk_count_dist_numpy

array([[0.85714286, 0.75714286, 0.        ],
       [0.9010989 , 0.91666667, 0.54545455],
       [0.10344828, 0.28571429, 0.79104478]])

In [27]:
assert (np.isclose(bulk_count_dist_numpy, 1 - bulk_count_sim_numpy)).all()

You can pass just one array, then the similarities will be computed between its rows.

In [28]:
X = np.array(
    [
        [1, 1, 1],
        [0, 0, 0],
        [1, 1, 1],
    ]
)

In [29]:
bulk_bin_sim_one_arr = bulk_tanimoto_binary_similarity(X)
bulk_bin_sim_one_arr

array([[1., 0., 1.],
       [0., 1., 0.],
       [1., 0., 1.]])

We expect that passing two copies of the same array will yield the same result
as passing only one copy.

In [30]:
bulk_bin_sim_two_arrs = bulk_tanimoto_binary_similarity(X, X)
bulk_bin_sim_two_arrs

array([[1., 0., 1.],
       [0., 1., 0.],
       [1., 0., 1.]])

In [31]:
assert np.allclose(bulk_bin_sim_one_arr, bulk_bin_sim_two_arrs)

Lastly, let's prove that bulk variant produces the same result as calling
a standard one in a nested loop. Lets use some real world data.

First of all, let's transform [smiles](https://en.wikipedia.org/wiki/Simplified_Molecular_Input_Line_Entry_System) into **fingerprints**, effectively turning them into vectors.

In [63]:
from skfp.fingerprints import ECFPFingerprint

mols_list = [
    "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # Ibuprofen
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # caffeine
    "c1ncccc1[C@@H]2CCCN2C",  # nicotine
    "C1CC1N2C=C(C(=O)C3=CC(=C(C=C32)N4CCNCC4)F)C(=O)O",  # Ciprofloxacin
    "CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O",  # Warfarin
    "CC(=O)Nc1ccc(O)cc1",  # Paracetamol
]

fp = ECFPFingerprint(count=True)
fps = fp.transform(mols_list)

fps[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint32)

Notice that the input vectors use **count** data. Let's call the appropriate method.

Calculate the similarity using nested loops.

In [64]:
manual_pairwise_sim = [
    [tanimoto_count_similarity(fps[i], fps[j]) for j in range(len(fps))]
    for i in range(len(fps))
]

And now, let's use a bulk function.

In [34]:
bulk_sim = bulk_tanimoto_binary_similarity(fps, fps)

In [35]:
assert np.allclose(manual_pairwise_sim, bulk_sim)

Let's also check the performance of both approaches.

In [65]:
%timeit -r 3 -n 10 [tanimoto_binary_similarity(fps[i], fps[j]) for i in range(len(fps)) for j in range(len(fps))]

4.47 ms ± 729 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [66]:
%timeit -r 3 -n 10 [bulk_tanimoto_binary_similarity(fps, fps)]

265 µs ± 68.2 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


Bulk variant is clearly faster.